In [ ]:
#r "nuget: MathNet.Numerics.FSharp, 4.15.0"

open MathNet.Numerics.LinearAlgebra

Installed package MathNet.Numerics.FSharp version 4.15.0

## Варинат 9

In [ ]:
let matrix = matrix [
    [-0.90701; -0.27716; 0.44570]
    [-0.27716; 0.63372; 0.07774]
    [0.44570; 0.07774; -0.95535]
]

1) Найти степенным методом c точностью ε = 0.001 максимальное по модулю собственное число λ1 матрицы A и соответствующий ему собственный вектор x_1, так чтобы ||x_1||_2 = 1

In [ ]:
let epsilon = 0.001

let findPowerMethod (matrix: Matrix<float>) = 
    let step (previous: Vector<float>) = 
        let y = matrix * previous
        let eigen = Vector.map2 (fun u v -> u / v) y previous
        y, eigen

    let mutable y = DenseVector.create matrix.RowCount 1.
    let mutable prevEigen = 0.
    let mutable currentEigen = 0.

    y <- fst <| step y
    currentEigen <- snd <| step y

    while (prevEigen - currentEigen).InfinityNorm() >= epsilon do
        prevEigen <- currentEigen
        y <- fst <| step y
        currentEigen <- snd <| step y
    
    currentEigen.max